# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [1]:
typeof(3)

Int64

In [2]:
typeof(3.0)

Float64

In [13]:
isconcretetype(Float64)

true

In [16]:
isabstracttype(Number)

true

In [6]:
isabstracttype(Real)

true

### Duck typing

A `Number` is some type that can do things like `+`,`-`,`*`, and `/`. In this category we have things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [18]:
supertype(Float64)

AbstractFloat

In [19]:
supertype(AbstractFloat)

Real

In [20]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [4]:
supertype(Real)

Number

In [5]:
supertype(Number)

Any

Everything is a subtype of Any

In [6]:
Number <: Any

true

In [7]:
Float64 <: Any

true

In [8]:
Int32 <: Any

true

In [9]:
Int32 <: String

false

There is also `isa` for objects:

In [10]:
3.0 isa Float64

true

In [11]:
3 isa Float64

false

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [22]:
function show_supertypes(T) 
 print(T)
 while T != Any 
     T = supertype(T) 
     print(" <: ", T) 
 end 
end

show_supertypes (generic function with 1 method)

In [23]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [24]:
show_supertypes(String)

String <: AbstractString <: Any

Let's extract a bunch of branches

In [26]:
function show_subtypetree(T, level=1, indent=4)
   level == 1 && println(T)
   for s in subtypes(T)
     println(join(fill(" ", level * indent)) * string(s))
     show_subtypetree(s, level+1, indent)
   end
end

show_subtypetree (generic function with 3 methods)

In [27]:
show_subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


Note that concrete types are the leaves of the type tree.

Abstract types are nodes in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how* for a .

We can use the `::` operator to annotate function arguments with types.

In [44]:
myabs(x::Float64) = sign(x) * x

myabs (generic function with 1 method)

In [45]:
myabs(-4.32)

4.32

In [50]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [51]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [52]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [53]:
methods(myabs)

# 2 methods for generic function "myabs":
[1] myabs(z::Complex{Float64}) in Main at In[51]:1
[2] myabs(x::Float64) in Main at In[44]:1

One can check which particular method is being used through the `@which` macro.

In [54]:
@which myabs(-4.32)

myabs(x::Float64) in Main at In[44]:1

In [55]:
@which myabs(1.0 + 1.0im)

myabs(z::Complex{Float64}) in Main at In[51]:1

Note that we should better loosen our type restrictions:

In [56]:
myabs(-3)

MethodError: MethodError: no method matching myabs(::Int64)
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[51]:1
  myabs(!Matched::Float64) at In[44]:1

In [57]:
myabs(1 + 1im)

MethodError: MethodError: no method matching myabs(::Complex{Int64})
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[51]:1
  myabs(!Matched::Float64) at In[44]:1

In [58]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 4 methods)

In [60]:
myabs(-3)

3

# Multiple Dispatch

In [40]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [42]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[40]:5
[2] f(a::Number, b::Number) in Main at In[40]:2
[3] f(a::Number, b) in Main at In[40]:3
[4] f(a, b::Number) in Main at In[40]:4
[5] f(a, b) in Main at In[40]:1

In [43]:
f(1.5, 2)

"a and b are both numbers"

In [44]:
f(1, "Köln!")

"a is a number"

In [45]:
f(1, 2)

"a and b are both integers"

In [46]:
f("Hello", "World!")

"fallback"

**Julia's dispatch mechanism always chooses the most specific method for the given input types.**

In [49]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[40]:5

In [53]:
@which f(1, "Köln!")

f(a::Number, b) in Main at In[40]:3

In [26]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:349
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:347
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\periods.jl:385
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Dates\src\arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\s

In [56]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:104

In [60]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:229

In [61]:
methodswith(Bool)

[1] sparsevec(I::AbstractArray{T,1} where T, V::Union{Bool, AbstractArray{Bool,1}}) in SparseArrays at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\SparseArrays\src\sparsevector.jl:217
[2] sparsevec(I::AbstractArray{T,1} where T, V::Union{Bool, AbstractArray{Bool,1}}, len::Integer) in SparseArrays at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\SparseArrays\src\sparsevector.jl:220
[3] process_messages(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool) in Distributed at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Distributed\src\process_messages.jl:107
[4] process_messages(r_stream::IO, w_stream::IO, incoming::Bool) in Distributed at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Distributed\src\process_messages.jl:135
[5] send(socket::ZMQ.Socket, data, more::Bool) in ZMQ at deprecated.jl:53
[6] !(x::Bool) in Base at bool.jl:35
[7] rem(x::Bool, y::Bool) in Base at bool.jl:125
[8] rem(x::Bool, ::Type{Int8}) in Base at int.jl:460
[9] rem(x::Bool, ::Type{Int16}) in Base at int.jl:460
[10] rem(x::Bool, ::Type{Int32}) in Base at int.jl:460
[11] rem(x::Bool, ::Type{Int64}) in Base at int.jl:460
[12] rem(x::Bool, ::Type{Int128}) in Base at int.jl:460
[13] rem(x::Bool, ::Type{UInt8}) in Base at int.jl:460
[14] rem(x::Bool, ::Type{UInt16}) in Base at int.jl:460
[15] rem(x::Bool, ::Type{UInt32}) in Base at int.jl:460
[16] rem(x::Bool, ::Type{UInt64}) in Base at int.jl:460
[17] rem(x::Bool, ::Type{UInt128}) in Base at int.jl:460
[18] &(b::Bool, a::Missing) in Base at missing.jl:125
[19] &(x::Bool, y::Bool) in Base at bool.jl:40
[20] &(a::Missing, b::Bool) in Base at missing.jl:124
[21] *(x::Bool, z::Complex{Bool}) in Base at complex.jl:281
[22] *(x::Bool, y::Bool) in Base at bool.jl:106
[23] *(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:118
[24] *(x::Bool, z::Complex) in Base at complex.jl:288
[25] *(x::Bool, y::AbstractIrrational) in Base at irrationals.jl:135
[26] *(z::Complex{Bool}, x::Bool) in Base at complex.jl:282
[27] *(y::AbstractFloat, x::Bool) in Base at bool.jl:120
[28] *(z::Complex, x::Bool) in Base at complex.jl:289
[29] *(A::LinearAlgebra.Symmetric, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[30] *(A::LinearAlgebra.Hermitian, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[31] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[32] +(x::Bool, y::Bool) in Base at bool.jl:104
[33] +(x::Bool) in Base at bool.jl:101
[34] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[35] +(x::Bool, z::Complex) in Base at complex.jl:284
[36] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[37] +(y::AbstractFloat, x::Bool) in Base at bool.jl:114
[38] +(z::Complex, x::Bool) in Base at complex.jl:285
[39] -(x::Bool, z::Complex{Bool}) in Base at complex.jl:279
[40] -(x::Bool, y::Bool) in Base at bool.jl:105
[41] -(x::Bool) in Base at bool.jl:102
[42] -(x::Bool, z::Complex) in Base at complex.jl:286
[43] -(z::Complex{Bool}, x::Bool) in Base at complex.jl:280
[44] -(z::Complex, x::Bool) in Base at complex.jl:287
[45] /(A::LinearAlgebra.Symmetric, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[46] /(A::LinearAlgebra.Hermitian, x::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\symmetric.jl:445
[47] <(x::Bool, y::Bool) in Base at bool.jl:96
[48] <<(x::Bool, c::Int64) in Base at bool.jl:82
[49] <<(x::Bool, c::Unsigned) in Base at bool.jl:78
[50] <<(x::Bool, c::Integer) in Base at bool.jl:8

It happens rarely, but it can happen that there is no unique most specific method:

In [19]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

MethodError: MethodError: f(::Int64, ::String) is ambiguous. Candidates:
  f(x, y::String) in Main at In[15]:2
  f(x::Int64, y) in Main at In[19]:1
Possible fix, define
  f(::Int64, ::String)

# Union types and `where`

TODO

# "Diagonal" dispatch

In [133]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [134]:
d(3, 4)

"same type"

In [135]:
d(3.0, 1.0)

"same type"

In [137]:
d(1, 4.2)

"different types"

# Specialization and code inspection

Internally, the compiler generates specialized code for particular input types.

When a function is called for the first time, julia compiles a specific version of the function for the given input types.

There are multiple stages and we can look into all of them using a bunch of macros:

* The AST after parsing <- Macros (`@macroexpand`)
* The AST after lowering (`@code_typed`, `@code_warntype`)
* The AST after type inference and optimization <- Generated Functions (`@code_lowered`)
* The LLVM IR <- Functions (`@code_llvm`)
* The assembly code (`@code_native`)

AST = Abstract Syntax Tree

In [1]:
myadd(x,y) = x + y

myadd (generic function with 1 method)

In [4]:
dump(:(myadd(3,4)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol myadd
    2: Int64 3
    3: Int64 4


In [67]:
@time myadd(1,1)

  0.002392 seconds (756 allocations: 44.626 KiB)


2

In [68]:
@time myadd(1,1)

  0.000002 seconds (4 allocations: 160 bytes)


2

In [69]:
@time myadd(1,1)

  0.000002 seconds (4 allocations: 160 bytes)


2

In [71]:
@code_typed myadd(1,1)

CodeInfo(
│╻ +1 1 ─ %1 = (Base.add_int)(x, y)::Int64
│   └──      return %1
) => Int64

In [72]:
@code_lowered myadd(1,1)

CodeInfo(
│1 1 ─ %1 = x + y
│  └──      return %1
)

In [73]:
@code_llvm myadd(1,1)


; Function myadd
; Location: In[66]:1
; Function Attrs: uwtable
define i64 @julia_myadd_35687(i64, i64) #0 {
top:
; Function +; {
; Location: int.jl:53
  %2 = add i64 %1, %0
;}
  ret i64 %2
}


In [18]:
@code_native myadd(1,1)

	.text
; Function myadd {
; Location: In[13]:1
	pushq	%rbp
	movq	%rsp, %rbp
; Function +; {
; Location: int.jl:53
	leaq	(%rcx,%rdx), %rax
;}
	popq	%rbp
	retq
	nopw	(%rax,%rax)
;}


Let's compare with `Float64` input.

In [17]:
@code_native myadd(1.0, 2.0)

	.text
; Function myadd {
; Location: In[13]:1
	pushq	%rbp
	movq	%rsp, %rbp
; Function +; {
; Location: float.jl:395
	vaddsd	%xmm1, %xmm0, %xmm0
;}
	popq	%rbp
	retq
	nopw	(%rax,%rax)
;}


## Specialization is important!

Let's try to estimate the performance of our `myadd` function if julia wouldn't specialize. We mimic this situation by wrapping our floating point numbers into a custom type which internally stores them as `Any`s.

In [12]:
struct Anything
    value::Any
end

add(x::Number,y::Number) = x + y
add(x::Anything,y::Anything) = x.value + y.value

add (generic function with 2 methods)

In [15]:
@time add(1, 2);
@time add(1.0, 2.0);

x = Anything(1.0)
y = Anything(2.0)
@time add(x,y);

  0.000002 seconds (4 allocations: 160 bytes)
  0.000002 seconds (5 allocations: 176 bytes)
  0.000002 seconds (5 allocations: 176 bytes)


Oh, seems to be equally fast. Screw specialization.

**Benchmarking isn't trivial!** There are tools in Julia that help you avoid the most common mistakes.

### Interlude: BenchmarkTools.jl

In [34]:
using BenchmarkTools

In [20]:
x = rand(2,2)
@time zero(x)
@time zero(x)

  0.010022 seconds (12.60 k allocations: 671.829 KiB)
  0.000002 seconds (5 allocations: 272 bytes)


2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [21]:
@time zero(1)
@time zero(1)

  0.000009 seconds (5 allocations: 240 bytes)
  0.000002 seconds (4 allocations: 160 bytes)


0

This must be faster...

In [39]:
@benchmark zero(x)

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  1
  --------------
  minimum time:     37.325 ns (0.00% GC)
  median time:      38.880 ns (0.00% GC)
  mean time:        49.817 ns (13.54% GC)
  maximum time:     32.641 μs (99.79% GC)
  --------------
  samples:          10000
  evals/sample:     994

In [40]:
@benchmark zero(1)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.001 ns (0.00% GC)
  median time:      0.001 ns (0.00% GC)
  mean time:        0.025 ns (0.00% GC)
  maximum time:     14.223 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

That make more sense!

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [46]:
@btime zero(x);
@btime zero(1);

  37.870 ns (1 allocation: 112 bytes)
  0.001 ns (0 allocations: 0 bytes)


Some more features

In [51]:
@btime zero($x); # interpolate the value of x into the expression to avoid overhead of globals

  25.970 ns (1 allocation: 112 bytes)


In [50]:
@btime zero(x) setup=(x=rand(2,2));

  25.662 ns (1 allocation: 112 bytes)


See [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md) for more information.

### Back to benchmarking Specialization

In [52]:
@btime add(1, 2);
@btime add(1.0, 2.0);

x = Anything(1.0)
y = Anything(2.0)
@btime add($x,$y);

  0.001 ns (0 allocations: 0 bytes)
  0.001 ns (0 allocations: 0 bytes)
  21.024 ns (1 allocation: 16 bytes)


**That's about 20000 times slower!**

## Explicit typing

Note that Julia's type inference is powerful. Specifying types isn't necessary for best performance!

In [92]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

my_function_typed (generic function with 1 method)

In [93]:
@btime my_function(10);
@btime my_function_typed(10);

  6.492 ns (0 allocations: 0 bytes)
  6.492 ns (0 allocations: 0 bytes)


 However it can serve one of the following purposes

* **Define a user interface** (will error if incompatible type is given)
* Enforce conversions
* Help the compiler infer types in tricky situations

In [64]:
add_first_two(x) = x[1] + x[2]

add_first_two (generic function with 1 method)

In [70]:
add_first_two(1:10)

3

In [66]:
add_first_two(3)

BoundsError: BoundsError

In [67]:
add_first_two_better(x::AbstractArray) = x[1] + x[2]

add_first_two_better (generic function with 1 method)

In [69]:
add_first_two_better(3) # better error message

MethodError: MethodError: no method matching add_first_two_better(::Int64)
Closest candidates are:
  add_first_two_better(!Matched::AbstractArray) at In[67]:1

In [73]:
add_first_two_better(split("Das ist ein Test!"))

MethodError: MethodError: no method matching +(::SubString{String}, ::SubString{String})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [74]:
typeof(split("Das ist ein Test!"))

Array{SubString{String},1}

To make an even preciser interface we have to learn about parametric types first.

# Parametric types

Types can have a nested structure.

In [138]:
typeof(rand(2,2))

Array{Float64,2}

In [139]:
typeof([1 2; 3 4])

Array{Int64,2}

In [76]:
eltype(rand(2,2))

Float64

In [140]:
typeof((1,2.0))

Tuple{Int64,Float64}

In [144]:
typeof([(1,2.0), (4,5.0)])

Array{Tuple{Int64,Float64},1}

In [149]:
Array{Float64,2} <: Array

true

In [166]:
Matrix{Float64} === Array{Float64, 2}

true

Note that parametric types have the following (somewhat counterintuitive) property

In [169]:
Array{Float64,2} <: Array{AbstractFloat,2}

false

although we have

In [170]:
Float64 <: AbstractFloat

true

The correct way to write this is

In [211]:
Array{Float64, 2} <: (Array{T, 2} where T<:AbstractFloat)

true

or shorter

In [213]:
Array{Float64, 2} <: Array{<:AbstractFloat, 2}

true

This is equivalent because we have

In [214]:
Array{<:AbstractFloat, 2} == (Array{T, 2} where T<:AbstractFloat)

true

**Quick exercise**: write a function that only takes real Matrices as input.

The following **won't work**:

In [5]:
g(x::Matrix{Real}) = "that was a matrix of real numbers"
g(x) = "wrong"

g (generic function with 3 methods)

In [6]:
g(rand(2,2))

"wrong"

In [7]:
g(rand(ComplexF64, 2,2))

"wrong"

The correct way is

In [8]:
g(x::Matrix{<:Real}) = "that was a matrix of real numbers"

# or

g(x::Matrix{T}) where T<:Real = "that was a matrix of real numbers"

g (generic function with 4 methods)

In [9]:
g(rand(2,2))

"that was a matrix of real numbers"

In [10]:
g(rand(ComplexF64, 2,2))

"wrong"

### Coming back to our add_first_two example

**Quick exercise**: define `add_first_two_even_better` as a refined version of `add_first_two_better`. It should take a reasonable subset of all `AbstractArrays`.

# Duck typing examples

## UnitRange

In [77]:
x = 1:30

1:30

In [80]:
typeof(x)

UnitRange{Int64}

In [81]:
typeof(x) <: AbstractArray

true

Because it is a subtype of AbstractArray I can do (some) array-like things with it, like indexing

In [82]:
x[3]

3

However, it's not a regular `Array`. In fact, it's just two numbers! We can see this by looking at it's fields:

In [93]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code (tracing the constructor)

In [94]:
@which UnitRange{Int64}(1, 10)

(::Type{UnitRange{T}})(start, stop) where T<:Real in Base at range.jl:255

It is an `immutable` type which just holds the start and stop values. This means that its indexing, `A[i]`, is just a function. What's nice about this is that means that no array is ever created. Creating large arrays can be a costly action:

In [96]:
@time collect(1:10000000);

  0.037605 seconds (7 allocations: 76.294 MiB, 26.12% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [98]:
@time 1:10000000;

  0.000001 seconds (5 allocations: 192 bytes)


Yet, in cases where we just want to index values, they act exactly the same.

## Uniform scaling operator

Another great example is the `UniformScaling` operator. It automatically gets loaded into scope when you do `using LinearAlgebra` and has the name `I`.

In [19]:
using LinearAlgebra

In [20]:
I

UniformScaling{Bool}
true*I

In [22]:
?I

search: I IO In if Int in im Inf isa Int8 inv Int64 Int32 Int16 imag Inf64 Inf32



```
I
```

An object of type [`UniformScaling`](@ref), representing an identity matrix of any size.

# Examples

```jldoctest
julia> fill(1, (5,6)) * I == fill(1, (5,6))
true

julia> [1 2im 3; 1im 2 3] * I
2×3 Array{Complex{Int64},2}:
 1+0im  0+2im  3+0im
 0+1im  2+0im  3+0im
```


Although it never actually allocates a full identity matrix it behaves like one

In [23]:
A = rand(1:10, 2,2)

2×2 Array{Int64,2}:
 10  8
  2  5

In [24]:
I * A

2×2 Array{Int64,2}:
 10  8
  2  5

In [25]:
A + I

2×2 Array{Int64,2}:
 11  8
  2  6

This can calculate expressions like `A-b*I` without ever forming the matrix `eye(n)` which would take $\mathcal{O}(n^2)$ memory. Let's benchmark the performance difference!

In [204]:
using BenchmarkTools
b = 3

@btime $A - $b * $I

  30.421 ns (1 allocation: 112 bytes)


2×2 Array{Int64,2}:
 5  5
 8  6

In [205]:
eye = Matrix(1.0I, 2,2) # alternatively but slower, diagm(0 => [1.0, 1.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

In [206]:
@btime $A - $b * Matrix(1.0I, 2,2)

  120.085 ns (3 allocations: 336 bytes)


2×2 Array{Float64,2}:
 5.0  5.0
 8.0  6.0

# Own type hierarchy: a slightly bigger example

This is the key idea to keep in mind when building type hierarchies: things which subtype are inheriting behavior. You should setup your `abstract` types to mean the existance or non-existance of some behavior. For example: 

In [41]:
abstract type AbstractPerson end
abstract type AbstractStudent <: AbstractPerson end
abstract type AbstractTeacher <: AbstractPerson end

mutable struct Person <: AbstractPerson
  name::String    
end

mutable struct Student <: AbstractStudent
  name::String  
  grade::Int
  hobby::String
end

mutable struct MusicStudent <: AbstractStudent
  grade::Int
end

mutable struct Teacher <: AbstractTeacher
  name::String
  grade::Int
end

This can be interpreted as follows. At the top we have `AbstractPerson`. Our interface here is "a Person is someone who has a name which can be gotten by `get_name`". 

In [42]:
get_name(x::AbstractPerson) = x.name

get_name (generic function with 1 method)

Thus codes which are written for an `AbstractPerson` can "know" (by our informal declaration of the interface) that `get_name` will "just work" for its subtypes. However, notice that `MusicStudent` doesn't have a `name` field. This is because `MusicStudent`s just want to be named whatever the trendiest band is, so we can just replace the usage of the field by the action:

In [43]:
get_name(x::MusicStudent) = "Die Höhner"

get_name (generic function with 2 methods)

In this way, we can use `get_name` to get the name, and how it was implemented (whether it's pulling something that had to be stored from memory, or if it's something magically known in advance) does not matter. We can keep refining this: an `AbstractStudent` has a `get_hobby`, but a `MusicStudent`'s hobby is always `Music`, so there's not reason to store that data in the type and instead just have its actions implicitly "know" this. In non-trivial examples (like the range and `UniformScaling` above), this distinction by action and abstraction away from the actual implementation of the types allows for full optimization of generic codes.

### Small functions and constant propagation

The next question to ask is, does storing information in functions and actions affect performance? The answer is yes, in a good way! To see this, let's see what happens when we use these functions. To make it simpler, let's use a boolean function. Teachers are old and don't like music, while students do like music. But generally people like music. This means that:

In [36]:
likes_music(x::AbstractTeacher) = false
likes_music(x::AbstractStudent) = true
likes_music(x::AbstractPerson) = true

likes_music (generic function with 3 methods)

Now how many records would these people buy at a record store? If they don't like music, they will buy zero records. If they like music, then they will pick up a random number between 1 and 10. If they are a student, they will then double that (impulsive Millenials!).

In [35]:
function number_of_records(x::AbstractPerson)
    if !likes_music(x) 
      return 0
    end
    num_records = rand(10)
    if typeof(x) <: AbstractStudent
      return 2num_records
    else 
      return num_records
    end
end

number_of_records (generic function with 1 method)

Let's check the code that is created:

In [41]:
x = Teacher("Randy",11)
println(number_of_records(x))
@code_llvm number_of_records(x)

0

; Function number_of_records
; Location: In[35]:2
; Function Attrs: uwtable
define i64 @julia_number_of_records_35125(%jl_value_t addrspace(10)* nonnull align 8 dereferenceable(16)) #0 {
top:
  ret i64 0
}


The key thing to see from the typed code is that the "branches" (the `if` statements) all compiled away. Since types are known at compile time (remember, functions specialize on types), the dispatch of `likes_music` is known at compile-time. But this means, since the result is directly inferred from the dispatch, the boolean value `true/false` is known at compile time. This means that the compiler can directly infer the answer to all of these checks, and will use this information to skip them at runtime.

This is the distinction between compile-time information and runtime information. **At compile-time, what is known is:**

**1) The types of the inputs**

**2) Any types which can be inferred from the input types (via type-stability)**

**3) The function dispatches that will be internally called (from types which have been inferred)**

Note that what cannot be inferred by the compiler is the information in fields. Information in fields is strictly runtime information. This is easy to see since there is no way for the compiler to know that person's name was "Miguel": it is part of the type instance we just created.

Thus by putting our information into our functions and dispatches, we are actually giving the compiler more information to perform more optimizations. Therefore using this "action-based design", we are actually giving the compiler leeway to perform many extra optimizations on our code as long as we define our interfaces by the actions that are used. Of course, at the "very bottom" our algorithms have to use the fields of the types, but the full interface can then be built up using a simple set of functions which in many cases with replace runtime data with constants.

# Tim Holy's Trait Trick (advanced)

Since traits like "likes music" are compile-time information the compiler could in theory dispatch on them. Imagine you'd want to write a function
```julia
entertain(p::LikesMusic) = "turning the music on."
entertain(p::NotLikesMusic) = "better keep the music off. maybe play a game."
```
A way to do this is Tim Holy's Trait Trick. It's basically a **traits-based alternative to multiple inheritance**.

The trick was "invented" by Tim Holy in this [github issue](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633). See https://github.com/mauro3/Traits.jl#dispatch-on-traits for a detailed explanation and [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) for a convenience implementation.

Before we can understand THTT we have to understand what `Type{SomeType}` is and what `f(::Type{SomeType}) = "some function"` means. 

**Find out yourself by playing around with it (`isa` is your friend).**

[Solution](https://docs.julialang.org/en/v1/manual/types/#man-singleton-types-1)

In [57]:
# Music affinity
struct LikesMusic end
struct NotLikesMusic end

# trait function: map, say, person type to music affinity
likes_music(x::AbstractTeacher) = NotLikesMusic
likes_music(x::AbstractStudent) = LikesMusic
likes_music(x::AbstractPerson) = LikesMusic
likes_music(x::T) where T = error("Unknown music affinity for type $T")

likes_music (generic function with 4 methods)

No let's define the function `entertain` which dispatches on the "likes music" trait.

In [65]:
_entertain(::Type{LikesMusic}) = "turning the music on."
_entertain(::Type{NotLikesMusic}) = "better keep the music off. maybe play a game."

entertain(p) = _entertain(likes_music(p))

entertain (generic function with 2 methods)

In [59]:
c = Student("Susanne", 11, "soccer")
p = Person("Peter")
t = Teacher("Thomas", 10)
m = MusicStudent(9)

MusicStudent(9)

In [63]:
@code_typed entertain(c)

CodeInfo(
│4 1 ─     return "turning the music on."
) => String

In [64]:
@code_typed entertain(t)

CodeInfo(
│4 1 ─     return "better keep the music off. maybe play a game."
) => String

In [68]:
entertain(123.45)

ErrorException: Unknown music affinity for type Float64

Note that this trait system is general, it works for non boolean traits as well.

**The most important thing is however, that it is easily extendable.**

As we all know, dogs love music as well! Note that the only thing we have to do to make `entertain` work for them is to define their "likes music" trait function:

In [73]:
abstract type Animals end

mutable struct Dog <: Animals # single inheritance
    name::String
end

likes_music(x::Dog) = LikesMusic # has the "likes music" trait defined

ErrorException: invalid redefinition of constant Dog

In [74]:
wuff = Dog("Wolfgang")
entertain(wuff)

"turning the music on."

Ok, but this is students, teachers, and dogs. What about physics? Where could I possibly need this?

Think of 

* `entertain` as `perform_matrix_operation`
* `likes_music` as `select_method`.
* `LikesMusic` could be `FastMethod`.
* `NoLikesMusic` could be `SlowMethod`.

# Other types

* Union types: `Union{Float64, Int32}`
* [Bitstypes](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#man-bits-types-1) (check with `isbits(x)`, `isbitstype(T)`)
* [Value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) (allows dispatch on values)

# Most important take home messages

* **Use types to move runtime information to compile-time.**
* **Use types and multiple dispatch to move `if` branches to compile time (and potentially compile them away). In my opinion, this also makes the code more readable.**

# Extra: slurping and splatting

Define a function that takes an **arbitrary number of input arguments** which does the following:
* `println` the first argument.
* return a concatenation of string representations of **all other arguments**